In [282]:
# Инициируем сессию
from pyspark.sql import SparkSession

spark = (SparkSession.builder
.master("local[*]")
.appName('PySpark_Tutorial')
.getOrCreate()
)

In [283]:
# Читаем JSON файл со странами
countries_json_file = '../Countries-Cities/world_population.json'
df_countries = spark.read.option('header', True).json(countries_json_file)

In [284]:
# Читаем CSV файл с городами
cities_csv_file = '../Countries-Cities/world_cities.csv'
df_cities = spark.read.csv(cities_csv_file, header=True, sep=";")

df_cities = df_cities.select(
    f.trim(f.col("city_ascii")).alias("city"),
    f.expr("lat").cast("decimal(10,4)").alias("lat"),
    f.expr("lng").cast("decimal(10,4)").alias("lng"),
    f.trim(f.col("country")).alias("country"),
    f.trim(f.col("capital")).alias("capital"),
    f.trim(f.col("population")).alias("population"),
    f.trim(f.col("id")).alias("id")                            
)

In [285]:
# Читаем CSV файл со справочником городов
codes_csv_file = '../Countries-Cities/CountryCodes.csv'
df_codes = spark.read.option('header', True).csv(codes_csv_file)

In [286]:
# Подготавливаем датафрейм со странами
from pyspark.sql import functions as f
countryDf = df_countries.select(
    f.trim(f.col("CCA3")).alias("CCA3"),
    f.trim(f.col("Capital")).alias("Capital"),
    f.trim(f.col("Continent")).alias("Continent"),       
    f.expr("`Area (sq mi)` * 2.59").cast("decimal(10,4)").alias("Area (km²)"),
    f.trim(f.col("Population")).cast("decimal(10,0)").alias("Population"),        
    f.expr("`Density (per sq mi)` * 0.3861").cast("decimal(10,4)").alias("Density (per km²)")                            
)

In [287]:
# Подготавливаем датафрейм с городами по странам
citiesDf = df_cities.fillna(0).groupBy("country") \
    .agg(f.count("city").alias("cities_count"), f.sum("population").cast("decimal").alias("urban_population"))

In [302]:
# Готовим результирующую таблицу
resultDf = countryDf \
        .join(df_codes, f.upper(countryDf["CCA3"]) == df_codes["code"], "left_outer") \
        .join(citiesDf, df_codes["name"] == citiesDf["country"]) \
        .withColumn("nonurban_population", f.col("Population") - f.col("urban_population")) \
        .drop("CCA3", "name")


In [305]:
df_countries.count()

234

In [ ]:
resultDf.write.save('../Countries-Cities/result.csv', format='csv')

In [ ]:
spark.stop()